# Traduciendo CUDA-C a PyCUDA

Dado que los kernels de PyCUDA siguen siendo código CUDA-C, es posible traducir los programas que hemos realizado previamente a PyCUDA de forma sencilla.

### vector_add_parallel.cu

``` 
#define N 400000000
#include <iostream>
#include <math.h>

__global__ void vector_add(float *out, float *a, float *b, int n) {
    int indice = threadIdx.x;//Indice del thread que ejecuta el kernel
    int paso = blockDim.x;//El numero de threads por bloque
    for(int i = indice; i < n; i+=paso){
        out[i] = a[i] + b[i];
    }
}

int main(){
    float *a, *b, *out; //Apuntadores a memoria del anfitrión
    float *cuda_a, *cuda_b, *cuda_out; //Apuntadores a memoria del GPU
    //Generamos los arreglos en memoria del GPU
    cudaMalloc((void**)&cuda_a, sizeof(float) * N);
    cudaMalloc((void**)&cuda_b, sizeof(float) * N);
    cudaMalloc((void**)&cuda_out, sizeof(float) * N);
    //Generamos los arreglos en memoria del anfitrión
    a   = (float*)malloc(sizeof(float) * N);
    b   = (float*)malloc(sizeof(float) * N);
    out = (float*)malloc(sizeof(float) * N);
    // Inicializamos a y b
    for(int i = 0; i < N; i++){
        a[i] = 1.0f; b[i] = 2.0f;
    }
    //Copiamos los vectores a y b al GPU.
    cudaMemcpy(cuda_a, a, sizeof(float) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_b, b, sizeof(float) * N, cudaMemcpyHostToDevice);
    // Llamamos al kernel de CUDA (1 bloque, 256 threads por bloque).
    vector_add<<<1,256>>>(cuda_out, cuda_a, cuda_b, N);
    //Copiamos el vector de salida del GPU al anfitrión.
    cudaMemcpy(out, cuda_out, sizeof(float) * N, cudaMemcpyDeviceToHost);
    float maxError = 0.0f;
    for (int i = 0; i < N; i++)
        maxError = fmax(maxError, fabs(out[i]-3.0f));
    std::cout << "Max error: " << maxError << std::endl;
    cudaFree(cuda_a);
    cudaFree(cuda_b);
    cudaFree(cuda_out);
    free(a);
    free(b);
    free(out);
    //Sugiero ver el comando: nvprof ./vector_add_parallel
}
``` 

Ahora lo reescribiremos a PyCUDA.

In [1]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy

from pycuda.compiler import SourceModule

mod = SourceModule("""
__global__ void vector_add(float *out, float *a, float *b, int n) {
    int indice = blockIdx.x * blockDim.x + threadIdx.x;//Indice del thread que ejecuta el kernel
    int paso = blockDim.x * gridDim.x;//El numero de threads por bloque
    for(int i = indice; i < n; i+=paso){
        out[i] = a[i] + b[i];
    }
}
""")

vector_add = mod.get_function("vector_add")
a = numpy.ones(400000000).astype(numpy.float32)
b = 2*numpy.ones(400000000).astype(numpy.float32)

resultado_esperado = a+b

dest = numpy.zeros_like(a)

start = drv.Event()
stop = drv.Event()
start.record()

vector_add(
        drv.Out(dest), drv.In(a), drv.In(b), numpy.int32(400000000),
        block=(256,1,1), grid=(100,1))

stop.record()
stop.synchronize()
time = start.time_till(stop)
print("Tiempo: "+str(time) + " ms")

a_confirmacion = (resultado_esperado-dest).sum()
a_confirmacion

Tiempo: 1825.8001708984375 ms


0.0

## Ejemplo de utilización de Streams

Código original: 
``` 
#define N 100000000
#include <iostream>
#include <math.h>

__global__ void vector_add(float *out, float *a, float *b, int n_max, int offset) {
    int indice = offset + blockIdx.x * blockDim.x + threadIdx.x;//Indice del thread que ejecuta el kernel
    int paso = blockDim.x * gridDim.x;//El numero de threads por bloque
    for(int i = indice; i < n_max; i+=paso){
        out[i] = a[i] + b[i];
    }
}

int main(){
    float *a, *b, *out; //Apuntadores a memoria del anfitrión
    float *cuda_a, *cuda_b, *cuda_out; //Apuntadores a memoria del GPU
    //Generamos los arreglos en memoria del GPU
    cudaMalloc((void**)&cuda_a, sizeof(float) * N);
    cudaMalloc((void**)&cuda_b, sizeof(float) * N);
    cudaMalloc((void**)&cuda_out, sizeof(float) * N);
    //Generamos los arreglos en memoria del anfitrión
    a   = (float*)malloc(sizeof(float) * N);
    b   = (float*)malloc(sizeof(float) * N);
    out = (float*)malloc(sizeof(float) * N);
    // Inicializamos a y b
    for(int i = 0; i < N; i++){
        a[i] = 1.0f; b[i] = 2.0f;
    }
    
    float ms;
    cudaEvent_t startEvent, stopEvent, dummyEvent;
    cudaEventCreate(&startEvent);
    cudaEventCreate(&stopEvent);
    cudaEventCreate(&dummyEvent);
    //Versión con un solo stream.
    cudaEventRecord(startEvent,0);
    cudaMemcpy(cuda_a, a, sizeof(float) * N, cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_b, b, sizeof(float) * N, cudaMemcpyHostToDevice);
    vector_add<<<100, 256>>>(cuda_out, cuda_a, cuda_b, N, 0);
    cudaMemcpy(out, cuda_out, sizeof(float) * N, cudaMemcpyDeviceToHost);
    cudaEventRecord(stopEvent, 0);
    cudaEventSynchronize(stopEvent);
    cudaEventElapsedTime(&ms, startEvent, stopEvent);
    printf("Tiempo para un solo stream (ms): %f\n", ms);
    float maxError = 0.0f;
    for (int i = 0; i < N; i++)
        maxError = fmax(maxError, fabs(out[i]-3.0f));
    std::cout << "Max error: " << maxError << std::endl;
    //Versiones con múltiples streams - Iteraciones múltiples
    //Haremos 10 streams distintos.
    int n_streams = 10;
    cudaStream_t stream[n_streams];
    for (int i = 0; i < n_streams; i ++)
    {
        cudaStreamCreate(&stream[i]);
    }
    cudaEventRecord(startEvent,0);
    int streamSize = N / n_streams;
    int streamBytes = sizeof(float) * N / n_streams;
    for (int i = 0; i < n_streams; i ++) 
    {
        int offset = i * streamSize;
        cudaMemcpyAsync(&cuda_a[offset], &a[offset], streamBytes, cudaMemcpyHostToDevice, stream[i]);
        cudaMemcpyAsync(&cuda_b[offset], &b[offset], streamBytes, cudaMemcpyHostToDevice, stream[i]);
    }
    for (int i = 0; i < n_streams; ++i)
    {
        int offset = i * streamSize;
        vector_add<<<100, 256, 0, stream[i]>>>(cuda_out, cuda_a, cuda_b, offset+streamSize, offset);
    }
    for (int i = 0; i < n_streams; i ++) 
    {
        int offset = i * streamSize;
        cudaMemcpyAsync(&out[offset], &cuda_out[offset], streamBytes, cudaMemcpyDeviceToHost, stream[i]);
    }
    cudaEventRecord(stopEvent, 0);
    cudaEventSynchronize(stopEvent);
    cudaEventElapsedTime(&ms, startEvent, stopEvent);
    printf("Tiempo con múltiples streams - Separación de tareas(ms): %f\n", ms);
    maxError = 0.0f;
    for (int i = 0; i < N; i++)
        maxError = fmax(maxError, fabs(out[i]-3.0f));
    std::cout << "Max error: " << maxError << std::endl;
    //Versión con múltiples streams - Una iteración por stream.
    cudaEventRecord(startEvent,0);
    for (int i = 0; i < n_streams; i ++) 
    {
        int offset = i * streamSize;
        cudaMemcpyAsync(&cuda_a[offset], &a[offset], streamBytes, cudaMemcpyHostToDevice, stream[i]);
        cudaMemcpyAsync(&cuda_b[offset], &b[offset], streamBytes, cudaMemcpyHostToDevice, stream[i]);
        vector_add<<<100, 256, 0, stream[i]>>>(cuda_out, cuda_a, cuda_b, offset+streamSize, offset);
        cudaMemcpyAsync(&out[offset], &cuda_out[offset], streamBytes, cudaMemcpyDeviceToHost, stream[i]);
    }
    cudaEventRecord(stopEvent, 0);
    cudaEventSynchronize(stopEvent);
    cudaEventElapsedTime(&ms, startEvent, stopEvent);
    printf("Tiempo con múltiples streams - Unificación de tareas (ms): %f\n", ms);
    maxError = 0.0f;
    for (int i = 0; i < N; i++)
        maxError = fmax(maxError, fabs(out[i]-3.0f));
    std::cout << "Max error: " << maxError << std::endl;
    cudaFree(cuda_a);
    cudaFree(cuda_b);
    cudaFree(cuda_out);
    free(a);
    free(b);
    free(out);
    //Sugiero ver el comando: nvprof ./vector_add_parallel_multiblock
}
``` 

Debe notarse que la traducción no es tan simple, por lo que se tiene que modificar el kernel para tener un comportamiento similar.

In [2]:
import pycuda.autoinit
import pycuda.driver as drv
import numpy

from pycuda.compiler import SourceModule

n = 400000000

#Tomado de vector_add_parallel_multiblock
mod = SourceModule("""
__global__ void vector_add(float *out, float *a, float *b, int n) {
    int indice = blockIdx.x * blockDim.x + threadIdx.x;//Indice del thread que ejecuta el kernel
    int paso = blockDim.x * gridDim.x;//El numero de threads por bloque
    for(int i = indice; i < n; i+=paso){
        out[i] = a[i] + b[i];
    }
}
""")

vector_add = mod.get_function("vector_add")

n_streams = 50
stream_size = int(n/n_streams)

shape, dtype = (stream_size), numpy.float32

a = [drv.pagelocked_empty(shape=shape, dtype=dtype) for i in range(n_streams)]
b = [drv.pagelocked_empty(shape=shape, dtype=dtype) for i in range(n_streams)]
dest = [drv.pagelocked_empty(shape=shape, dtype=dtype) for i in range(n_streams)]

a_cuda = []
b_cuda = []
dest_cuda = []

for i in range(n_streams):
    a[i][:] = 1.0
    b[i][:] = 2.0
    a_cuda.append(drv.mem_alloc(a[0].nbytes))
    b_cuda.append(drv.mem_alloc(b[0].nbytes))
    dest_cuda.append(drv.mem_alloc(dest[0].nbytes))

streams = [drv.Stream() for x in range(n_streams)]
start = drv.Event()
stop = drv.Event()
start.record()
for i, stream in enumerate(streams):
    offset = i * stream_size
    
    drv.memcpy_htod_async(a_cuda[i], a[i], stream)
    drv.memcpy_htod_async(b_cuda[i], b[i], stream)
    
    vector_add(
        dest_cuda[i],
        a_cuda[i],
        b_cuda[i],
        numpy.int32(stream_size),
        block=(256,1,1),
        grid=(100,1),
        stream = stream)
    
    drv.memcpy_dtoh_async(dest[i], dest_cuda[i], stream)
    
drv.Context.synchronize()
stop.record()
stop.synchronize()
time = start.time_till(stop)
print("Tiempo: "+str(time) + " ms")

error = 0
for i in range(n_streams):
    error += dest[i].sum() - 3.0*len(dest[i])

print("Error: "+str(error))

Tiempo: 555.781982421875 ms
Error: 0.0


Como se puede ver, es menos transparente que incluso CUDA-C debido a la dificultad de usar direcciones de memoria directas. Sin embargo el tiempo de ejecución es idéntico al de CUDA-C (dentro del GPU, en Python claramente no).

# Uso de GPUArray
Una abstracción de PyCUDA es el GPUArray, que permite generar arreglos que vuelvan las operaciones vectorizadas transparentes.

In [6]:
import pycuda.gpuarray as gpuarray
import pycuda.driver as cuda
import pycuda.autoinit
import numpy

a = numpy.ones(100000000).astype(numpy.float32)*2
b = numpy.ones(100000000).astype(numpy.float32)

a_gpu = gpuarray.to_gpu(a)
b_gpu = gpuarray.to_gpu(b)

a_resultado = (a_gpu + b_gpu).get()
error = a_resultado.sum() - 100000000 * 3.0
print("Error: "+str(error))

MemoryError: cuMemAlloc failed: out of memory

Sin embargo, no es tan transparente y sencillo como parece.